In [5]:
#Import des librairies Python
import pandas as pd
from sklearn import preprocessing
import pickle


In [6]:
#Jeu de données à stocker dans le dossier "datas/…"
mydata = 'datas/test_example.csv'
#Aperçu rapide du jeu de données
pd.read_csv(mydata)

diagonal  height_left  height_right  margin_low  margin_up  length   id
0    171.76       104.01        103.54        5.21       3.30  111.42  A_1
1    171.87       104.17        104.13        6.00       3.31  112.09  A_2
2    172.00       104.58        104.29        4.99       3.39  111.57  A_3
3    172.49       104.55        104.34        4.44       3.03  113.20  A_4
4    171.65       103.63        103.56        3.77       3.16  113.33  A_5

In [7]:
#Fonction "detecteur_billet()" à utiliser dans l'étape suivante
def detecteur_billet(mydata):
    """ Programme de détection de faux billets à partir d'un fichier .csv
        ici 'mydata' 
    """
    
    # Chargement et préparation des données 
    df = pd.read_csv(mydata)
    X = df.copy()
    X = X.iloc[:, :-1]
    
    # Centrage/réduction des données (éviter les écarts d'échelle, les données prendront toutes la même importance)
    std_scale = preprocessing.StandardScaler().fit(X)
    X_scaled = std_scale.transform(X)
    
    # Récupération du modèle de Régression Logistique 'lr'
    with open('datas/model.pkl', 'rb') as f:
        my_unpickler = pickle.Unpickler(f)
        lr = my_unpickler.load()
    
    # Utilisation du modèle de prédiction 'lr'
    probability = lr.predict_proba(X_scaled)[:, 1]
    
    # Probabilités des billets établies 
    proba = pd.Series(probability.round(3), name='%value')
    
    # Intégration des probabilités dans le jeu de données
    df_final = pd.concat([df, proba], axis=1)
    
    # Résultats de la classification prédictive, selon une probabilité  supérieure ou égale à 0.5 :
    resultat = []
    for i in df_final['%value'] >= .5:
        if i is True :
            resultat.append('Vrai Billet')
        else :
            resultat.append('Faux Billet')

    df_final['resultat'] = resultat

    return df_final
    
    

In [8]:
#Dernière étape avant l'affichage des prédictions 
detecteur_billet(mydata)

/opt/anaconda3/lib/python3.9/site-packages/sklearn/base.py:310: UserWarning: Trying to unpickle estimator LogisticRegression from version 0.22 when using version 0.24.2. This might lead to breaking code or invalid results. Use at your own risk.
  warnings.warn(


diagonal  height_left  height_right  margin_low  margin_up  length   id  \
0    171.76       104.01        103.54        5.21       3.30  111.42  A_1   
1    171.87       104.17        104.13        6.00       3.31  112.09  A_2   
2    172.00       104.58        104.29        4.99       3.39  111.57  A_3   
3    172.49       104.55        104.34        4.44       3.03  113.20  A_4   
4    171.65       103.63        103.56        3.77       3.16  113.33  A_5   

   %value     resultat  
0   0.111  Faux Billet  
1   0.007  Faux Billet  
2   0.003  Faux Billet  
3   0.948  Vrai Billet  
4   1.000  Vrai Billet